In [ ]:
# =============================================
# AUTOGLUON - ПОЛНЫЙ ПАЙПЛАЙН С ПОЯСНЕНИЯМИ
# =============================================

# Установка: pip install autogluon

# Импорт библиотек
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import time

"""
AutoGluon автоматически строит ансамбли моделей для табличных данных.
Он поддерживает три типа задач:
- binary classification (бинарная классификация)
- multiclass classification (многоклассовая классификация)  
- regression (регрессия)
"""

# ЗАГРУЗКА ДАННЫХ
data = pd.read_csv('your_data.csv')  # замените на ваш файл

"""
Предобработка данных - КРИТИЧЕСКИ ВАЖНЫЙ ЭТАП:
- AutoGluon сам обрабатывает пропуски, но лучше сделать это вручную
- Автоматически определяет типы признаков (числовые, категориальные)
- Нормализует числовые признаки
"""

# ПРЕДОБРАБОТКА ДАННЫХ
print("Форма данных до обработки:", data.shape)
data = data.drop_duplicates()  # удаляем дубликаты

# Обработка пропусков - несколько стратегий:
# data = data.fillna(method='ffill')  # заполнить предыдущим значением
# data = data.fillna(data.median())   # заполнить медианой для числовых
# data = data.fillna(data.mode().iloc[0])  # заполнить модой для категориальных
data = data.fillna(-999)  # специальное значение для пропусков

print("Форма данных после обработки:", data.shape)

# РАЗДЕЛЕНИЕ НА ПРИЗНАКИ И ЦЕЛЕВУЮ ПЕРЕМЕННУЮ
# Предполагаем, что целевая переменная называется 'target'
X = data.drop('target', axis=1)  # замените 'target' на имя вашей целевой переменной
y = data['target']

"""
Определяем тип задачи:
- binary: 2 класса (0 и 1)
- multiclass: больше 2 классов  
- regression: непрерывная числовая переменная
"""
problem_type = 'binary' if len(y.unique()) == 2 else 'multiclass'

# РАЗДЕЛЕНИЕ НА ОБУЧАЮЩУЮ И ТЕСТОВУЮ ВЫБОРКИ
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,           # 20% данных на тест
    random_state=42,         # для воспроизводимости
    stratify=y               # сохраняет распределение классов (только для классификации!)
)

# СОЗДАНИЕ ДАТАСЕТОВ ДЛЯ AUTOGLUON
train_data = TabularDataset(pd.concat([X_train, y_train], axis=1))
test_data = TabularDataset(pd.concat([X_test, y_test], axis=1))

"""
ДОСТУПНЫЕ МОДЕЛИ В AUTOGLUON:
- 'GBM': LightGBM (градиентный бустинг) - ОЧЕНЬ БЫСТРЫЙ И ТОЧНЫЙ
- 'CAT': CatBoost (отлично для категориальных признаков)
- 'XGB': XGBoost (классика бустинга)
- 'RF': Random Forest (случайный лес)
- 'XT': Extra Trees (экстра деревья)
- 'KNN': K-ближайших соседей
- 'LR': Линейная модель (логистическая регрессия)
- 'NN': Нейронная сеть
- 'FASTAI': Нейронная сеть из fastai

ДОСТУПНЫЕ МЕТРИКИ ДЛЯ КЛАССИФИКАЦИИ:
- 'accuracy' (точность)
- 'f1' (f1-score)
- 'roc_auc' (площадь под ROC-кривой)
- 'log_loss' (логистическая потеря)
- 'precision' (точность)
- 'recall' (полнота)

ДЛЯ РЕГРЕССИИ:
- 'mean_absolute_error' (MAE)
- 'mean_squared_error' (MSE) 
- 'root_mean_squared_error' (RMSE)
- 'r2' (R-квадрат)
"""

# НАСТРОЙКА ГИПЕРПАРАМЕТРОВ - САМАЯ ВАЖНАЯ ЧАСТЬ!
hyperparameters = {
    # LightGBM - обычно лучший баланс скорости и качества
    'GBM': {
        'num_boost_round': 1000,      # число деревьев (больше = точнее, но дольше)
        'num_leaves': 127,            # сложность деревьев (2^7-1)
        'learning_rate': 0.05,        # скорость обучения (меньше = точнее)
        'feature_fraction': 0.8,      # доля признаков для каждого дерева
    },
    
    # CatBoost - лучший для категориальных признаков
    'CAT': {
        'iterations': 1000,           # число итераций
        'depth': 8,                   # глубина деревьев
        'learning_rate': 0.05,        # скорость обучения
        'l2_leaf_reg': 3,             # регуляризация
    },
    
    # XGBoost - надежный классик
    'XGB': {
        'n_estimators': 1000,         # число деревьев
        'max_depth': 8,               # глубина деревьев
        'learning_rate': 0.05,        # скорость обучения
        'subsample': 0.8,             # доля данных для каждого дерева
    },
    
    # Random Forest - стабилен, не переобучается
    'RF': {
        'n_estimators': 200,          # число деревьев
        'max_depth': 15,              # глубина деревьев
        'criterion': 'gini',          # критерий разделения
    }
}

"""
PRESETS (предустановки) - выбирают баланс скорость/качество:
- 'best_quality': максимальное качество (очень долго)
- 'high_quality': высокое качество (долго)  
- 'good_quality': хорошее качество (среднее время)
- 'medium_quality_faster_train': среднее качество (быстро) - ЛУЧШИЙ ВЫБОР ДЛЯ ОЛИМПИАД
- 'optimize_for_deployment': оптимизировано для продакшена
"""

start_time = time.time()

# СОЗДАНИЕ И ОБУЧЕНИЕ ПРЕДСКАЗАТЕЛЯ
predictor = TabularPredictor(
    label='target',           # имя целевой переменной
    problem_type=problem_type,# тип задачи
    eval_metric='accuracy'    # метрика для оптимизации
).fit(
    train_data=train_data,
    time_limit=300,           # 5 минут - РЕГУЛИРУЙТЕ ПОД ЗАДАЧУ!
    presets='medium_quality_faster_train',  # баланс скорость/качество
    hyperparameters=hyperparameters,
    
    # ДОПОЛНИТЕЛЬНЫЕ ПАРАМЕТРЫ:
    # num_bag_folds=5,        # число фолдов для бэггинга (улучшает качество)
    # num_stack_levels=1,     # уровни стекинга (улучшает качество, но дольше)
    # auto_stack=True,        # автоматический стекинг
)

training_time = time.time() - start_time
print(f"AutoGluon обучение заняло: {training_time:.2f} секунд")

# ПРЕДСКАЗАНИЯ
y_pred = predictor.predict(test_data)              # предсказанные классы
y_pred_proba = predictor.predict_proba(test_data)  # вероятности классов

# ОЦЕНКА КАЧЕСТВА
leaderboard = predictor.leaderboard(test_data, silent=False)
print("ТОП-10 моделей:")
print(leaderboard.head(10))

"""
Leaderboard показывает:
- model: имя модели
- score_val: оценка на валидации (по выбранной метрике)
- pred_time_val: время предсказания
- fit_time: время обучения
- stack_level: уровень стекинга
"""

# СОХРАНЕНИЕ МОДЕЛИ
predictor.save('autogluon_model')

# ЗАГРУЗКА МОДЕЛИ (для последующего использования)
# predictor = TabularPredictor.load('autogluon_model')

# ПРЕДСКАЗАНИЯ НА НОВЫХ ДАННЫХ
# new_data = TabularDataset(pd.read_csv('new_data.csv'))
# predictions = predictor.predict(new_data)

In [ ]:
# =============================================
# H2O AUTOML - ПОЛНЫЙ ПАЙПЛАЙН С ПОЯСНЕНИЯМИ  
# =============================================

# Установка: pip install h2o

import h2o
from h2o.automl import H2OAutoML
import pandas as pd
import time

"""
H2O AutoML - промышленное решение для автоматического ML
Особенности:
- Работает через кластер H2O
- Отличная масштабируемость на больших данных
- Включает автоматический стекинг
- Поддерживает широкий спектр алгоритмов
"""

# ИНИЦИАЛИЗАЦИЯ H2O КЛАСТЕРА
h2o.init()

"""
Параметры h2o.init():
- nthreads: число потоков (-1 = все доступные)
- max_mem_size: максимальная память ('4G' = 4GB)
- port: порт для веб-интерфейса (54321 по умолчанию)
"""

# ЗАГРУЗКА ДАННЫХ
data = pd.read_csv('your_data.csv')

# ПРЕДОБРАБОТКА
print("Размер данных до обработки:", data.shape)
data = data.drop_duplicates().fillna(method='ffill')
print("Размер данных после обработки:", data.shape)

# КОНВЕРТАЦИЯ В H2O FRAME
hf = h2o.H2OFrame(data)

"""
H2O Frame - специальный формат данных для H2O
Автоматически определяет типы колонок:
- numeric: числовые
- categorical: категориальные
- string: строковые
- time: временные
"""

# ОПРЕДЕЛЕНИЕ ПРИЗНАКОВ И ЦЕЛЕВОЙ ПЕРЕМЕННОЙ
predictors = hf.columns[:-1]  # все колонки кроме последней
target = hf.columns[-1]       # последняя колонка - целевая

print(f"Признаки: {predictors}")
print(f"Целевая переменная: {target}")

"""
ДЛЯ КЛАССИФИКАЦИИ нужно преобразовать целевую в фактор:
- binary: 2 уровня
- multiclass: больше 2 уровней

ДЛЯ РЕГРЕССИИ оставляем как numeric
"""
# Определяем тип задачи автоматически
if hf[target].nlevels()[0] > 0 and hf[target].nlevels()[0] <= 10:  # категориальная
    hf[target] = hf[target].asfactor()
    problem_type = 'classification'
else:  # числовая - регрессия
    problem_type = 'regression'

print(f"Тип задачи: {problem_type}")

# РАЗДЕЛЕНИЕ НА TRAIN/TEST
train, test = hf.split_frame(ratios=[0.8], seed=42)

print(f"Размер train: {train.shape}")
print(f"Размер test: {test.shape}")

"""
ДОСТУПНЫЕ АЛГОРИТМЫ В H2O:
- "DRF": Distributed Random Forest
- "GBM": Gradient Boosting Machine (включая XGBoost-style)
- "XGBoost": Нативный XGBoost
- "GLM": Generalized Linear Model
- "DeepLearning": Нейронные сети
- "StackedEnsemble": Автоматический стекинг (всегда включен)

МЕТРИКИ ДЛЯ КЛАССИФИКАЦИИ:
- "AUC" (площадь под ROC-кривой)
- "logloss" (логистическая потеря)
- "accuracy" (точность)
- "precision" (точность)
- "recall" (полнота)
- "f1" (f1-score)

МЕТРИКИ ДЛЯ РЕГРЕССИИ:
- "deviance"
- "mse" (среднеквадратичная ошибка)
- "rmse" (корень из MSE)
- "mae" (средняя абсолютная ошибка)
- "rmsle" (RMSE логарифмическая)
- "r2" (R-квадрат)
"""

start_time = time.time()

# СОЗДАНИЕ И ЗАПУСК AUTOML
aml = H2OAutoML(
    # ОСНОВНЫЕ ПАРАМЕТРЫ:
    max_runtime_secs=300,           # 5 минут - ОСНОВНОЙ ЛИМИТ!
    max_models=20,                  # максимальное число моделей
    
    # СТРАТЕГИЯ ОСТАНОВКИ:
    stopping_metric="AUTO",         # автоматический выбор метрики
    stopping_rounds=3,              # ранняя остановка после 3 ухудшений
    stopping_tolerance=0.001,       # допуск для улучшения
    
    # ВАЛИДАЦИЯ:
    nfolds=5,                       # число фолдов кросс-валидации
    seed=42,                        # для воспроизводимости
    
    # ВЫБОР АЛГОРИТМОВ:
    include_algos=["GBM", "XGBoost", "GLM", "DRF", "DeepLearning"],
    # exclude_algos=["DeepLearning"],  # если нужно исключить
    
    # ДЛЯ ЭКСПЕРИМЕНТОВ:
    # exploitation_ratio=0.1,       # соотношение exploration/exploitation
    # modeling_plan=1,              # план моделирования
    # monotone_constraints=None,    # монотонные ограничения
)

# ОБУЧЕНИЕ
aml.train(
    x=predictors,                    # признаки
    y=target,                        # целевая переменная
    training_frame=train,            # обучающие данные
    validation_frame=test            # валидационные данные (опционально)
)

training_time = time.time() - start_time
print(f"H2O AutoML обучение заняло: {training_time:.2f} секунд")

# ПРОСМОТР РЕЗУЛЬТАТОВ
lb = aml.leaderboard
print("LEADERBOARD (ТОП-20 моделей):")
print(lb.head(20))

"""
Leaderboard показывает:
- model_id: идентификатор модели
- auc: AUC (для классификации)
- logloss: логистическая потеря
- mean_per_class_error: средняя ошибка по классам
- rmse: RMSE (для регрессии)
- mse: MSE (для регрессии)
"""

# ЛУЧШАЯ МОДЕЛЬ
leader_model = aml.leader
print(f"ЛУЧШАЯ МОДЕЛЬ: {leader_model.model_id}")

# ПРЕДСКАЗАНИЯ
predictions = aml.leader.predict(test)

# КОНВЕРТАЦИЯ ПРЕДСКАЗАНИЙ В PANDAS
pred_df = predictions.as_data_frame()
print("Пример предсказаний:")
print(pred_df.head())

"""
Для классификации predictions содержит:
- predict: предсказанный класс
- p0: вероятность класса 0
- p1: вероятность класса 1 (и т.д. для многоклассовой)

Для регрессии:
- predict: предсказанное значение
"""

# ВАЖНЫЕ МЕТРИКИ ЛУЧШЕЙ МОДЕЛИ
if problem_type == 'classification':
    performance = leader_model.model_performance(test)
    print(f"Accuracy: {performance.accuracy()}")
    print(f"AUC: {performance.auc()}")
    print(f"F1: {performance.F1()}")
else:
    performance = leader_model.model_performance(test)
    print(f"RMSE: {performance.rmse()}")
    print(f"MAE: {performance.mae()}")
    print(f"R2: {performance.r2()}")

# СОХРАНЕНИЕ МОДЕЛИ
model_path = h2o.save_model(
    model=aml.leader,
    path="./h2o_model",
    force=True
)
print(f"Модель сохранена в: {model_path}")

# ЗАГРУЗКА МОДЕЛИ (для последующего использования)
# loaded_model = h2o.load_model(model_path)

# ВЕБ-ИНТЕРФЕЙС H2O (дополнительно)
# Откройте в браузере: http://localhost:54321

# ЗАВЕРШЕНИЕ РАБОТЫ H2O
h2o.cluster().shutdown()

In [ ]:
# =============================================
# FLAML - ПОЛНЫЙ ПАЙПЛАЙН С ПОЯСНЕНИЯМИ
# =============================================

# Установка: pip install flaml

from flaml import AutoML
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error
import time

"""
FLAML (от Microsoft) - фокусируется на эффективном поиске гиперпараметров
Особенности:
- Очень быстрый поиск оптимальных параметров
- Эффективное использование времени
- Поддержка бюджетного обучения
- Хорош для ограниченных вычислительных ресурсов
"""

# ЗАГРУЗКА ДАННЫХ
data = pd.read_csv('your_data.csv')
X = data.drop('target', axis=1)  # замените 'target'
y = data['target']

# ПРЕДОБРАБОТКА
print("Размер данных до обработки:", X.shape)
X = X.fillna(method='ffill')  # обработка пропусков
print("Размер данных после обработки:", X.shape)

# ОПРЕДЕЛЕНИЕ ТИПА ЗАДАЧИ
if y.dtype == 'object' or len(y.unique()) <= 10:  # категориальная
    problem_type = 'classification'
    metric = 'accuracy'  # можно изменить на 'f1', 'roc_auc' и т.д.
else:  # числовая
    problem_type = 'regression' 
    metric = 'mae'  # можно изменить на 'mse', 'r2' и т.д.

print(f"Тип задачи: {problem_type}")
print(f"Метрика: {metric}")

# РАЗДЕЛЕНИЕ ДАННЫХ
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y if problem_type == 'classification' else None
)

"""
ДОСТУПНЫЕ АЛГОРИТМЫ В FLAML:
- 'lgbm': LightGBM (обычно лучший)
- 'xgboost': XGBoost
- 'catboost': CatBoost (требует установки catboost)
- 'rf': Random Forest
- 'extra_tree': Extra Trees
- 'kneighbor': K-Nearest Neighbors
- 'lrl1': LogisticRegression с L1 регуляризацией
- 'lrl2': LogisticRegression с L2 регуляризацией

МЕТРИКИ ДЛЯ КЛАССИФИКАЦИИ:
- 'accuracy', 'roc_auc', 'f1', 'micro_f1', 'macro_f1'
- 'log_loss', 'precision', 'recall'

МЕТРИКИ ДЛЯ РЕГРЕССИИ:
- 'mae', 'mse', 'rmse', 'r2', 'mape'
"""

# СОЗДАНИЕ AUTOML ОБЪЕКТА
automl = AutoML()

"""
НАСТРОЙКИ FLAML:
- time_budget: общее время в секундах (САМЫЙ ВАЖНЫЙ ПАРАМЕТР)
- metric: метрика для оптимизации
- task: тип задачи
- estimator_list: список алгоритмов для использования
- n_splits: число фолдов кросс-валидации
- eval_method: метод оценки ('auto', 'cv', 'holdout')
- log_file_name: файл для логов
- early_stop: ранняя остановка
"""

start_time = time.time()

# ОБУЧЕНИЕ
automl.fit(
    X_train=X_train, 
    y_train=y_train,
    
    # ОСНОВНЫЕ ПАРАМЕТРЫ:
    time_budget=300,              # 5 минут - БЮДЖЕТ ВРЕМЕНИ!
    task=problem_type,            # тип задачи
    metric=metric,                # метрика для оптимизации
    
    # ВЫБОР АЛГОРИТМОВ:
    estimator_list=['lgbm', 'xgboost', 'catboost', 'rf'],  # лучшие алгоритмы
    # estimator_list=['auto'],    # автоматический выбор
    
    # НАСТРОЙКИ ПОИСКА:
    n_splits=5,                   # число фолдов CV
    eval_method='auto',           # метод оценки
    seed=42,                      # для воспроизводимости
    log_file_name='flaml.log',    # файл логов
    
    # ДОПОЛНИТЕЛЬНЫЕ НАСТРОЙКИ:
    early_stop=True,              # ранняя остановка
    # log_type='all',             # детальное логирование
    # model_history=True,         # сохранять историю моделей
    # ensemble=True,              # включать ансамбли
)

training_time = time.time() - start_time
print(f"FLAML обучение заняло: {training_time:.2f} секунд")

# ПРЕДСКАЗАНИЯ
y_pred = automl.predict(X_test)

if problem_type == 'classification':
    y_pred_proba = automl.predict_proba(X_test)

# ОЦЕНКА КАЧЕСТВА
if problem_type == 'classification':
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Точность на тесте: {accuracy:.4f}")
else:
    mae = mean_absolute_error(y_test, y_pred)
    print(f"MAE на тесте: {mae:.4f}")

# ИНФОРМАЦИЯ О ЛУЧШЕЙ МОДЕЛИ
print(f"Лучший алгоритм: {automl.best_estimator}")
print(f"Лучшие параметры: {automl.best_config}")
print(f"Лучшая метрика на валидации: {automl.best_loss:.4f}")

"""
automl.best_config содержит оптимальные параметры:
- n_estimators: число деревьев
- max_leaves: максимальное число листьев
- learning_rate: скорость обучения
- и другие в зависимости от алгоритма
"""

# ДЕТАЛЬНАЯ ИНФОРМАЦИЯ О ВСЕХ ПОПЫТКАХ
print("\nИстория обучения:")
for config in automl.best_config_per_estimator:
    estimator = config['estimator']
    params = config['hyperparameters']
    loss = config['loss']
    print(f"{estimator}: loss={loss:.4f}, params={params}")

# СОХРАНЕНИЕ МОДЕЛИ
import pickle
with open('flaml_model.pkl', 'wb') as f:
    pickle.dump(automl, f)
print("Модель сохранена как 'flaml_model.pkl'")

# ЗАГРУЗКА МОДЕЛИ (для последующего использования)
# with open('flaml_model.pkl', 'rb') as f:
#     automl_loaded = pickle.load(f)

# ПРЕДСКАЗАНИЯ НА НОВЫХ ДАННЫХ
# new_predictions = automl.predict(new_data)

# ОПТИМИЗАЦИЯ ДЛЯ ОЛИМПИАДЫ:
"""
Стратегии для разных временных бюджетов:

1. МАЛО ВРЕМЕНИ (1-5 минут):
   - time_budget=60
   - estimator_list=['lgbm']  # только самый быстрый
   
2. СРЕДНЕЕ ВРЕМЯ (5-15 минут):
   - time_budget=600
   - estimator_list=['lgbm', 'xgboost', 'rf']
   
3. МНОГО ВРЕМЕНИ (30+ минут):
   - time_budget=1800
   - estimator_list=['lgbm', 'xgboost', 'catboost', 'rf']
   - n_splits=10  # больше фолдов
"""

# БЫСТРЫЙ СТАРТ ДЛЯ ОЛИМПИАДЫ (если очень мало времени):
"""
automl_fast = AutoML()
automl_fast.fit(
    X_train=X_train,
    y_train=y_train, 
    time_budget=60,           # всего 1 минута
    task=problem_type,
    metric=metric,
    estimator_list=['lgbm'],  # только LightGBM
    n_splits=3,               # меньше фолдов для скорости
)
"""